In [1]:
import os
import sys
from functools import lru_cache
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from svea_expvis.config import Config
from svea_expvis.readers.json_reader import load_json
from svea_expvis.bokeh_widgets.map import get_map
from svea_expvis.bokeh_widgets.callbacks import lasso_transect_callback, lasso_correlation_callback, update_colormapper_transect, update_colormapper_range
from svea_expvis.bokeh_widgets.utils import (
    get_columndata_source_json, 
    get_position_columndata_source_json, 
    get_matching_columndata_source,
    get_matching_positions,
    )

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, PreText, Select, Circle
from bokeh.layouts import column, row


DEFAULT_PARAMETERS = ['DOXY_CTD', 'DENS_CTD']

output_notebook()


Loading BokehJS ...

In [2]:
cfig = Config(data_file_name='ctd_data_sep2021.json')

data = load_json(cfig.data_path)
ctd_datasource = get_columndata_source_json(data, xy_params=True)
ctd_pos_source = get_position_columndata_source_json(data)

cfig.data_file_name = 'mvp_data_sep2021.json'

data = load_json(cfig.data_path)
mvp_datasource = get_columndata_source_json(data, xy_params=True)
mvp_pos_source = get_position_columndata_source_json(data)

matching_pos = get_matching_positions(data_ctd=ctd_pos_source, data_mvp=mvp_pos_source)
matching_datasource = get_matching_columndata_source(
    data_ctd=ctd_datasource,
    data_mvp=mvp_datasource,
    match_timestrings=matching_pos,
)

profiles_ctd_source = ColumnDataSource(data=dict(x=[], y=[], z=[]))
profiles_mvp_source = ColumnDataSource(data=dict(x=[], y=[], z=[]))

del data

In [3]:
# mvp_pos_source.data['lat']


In [4]:

"""
https://docs.bokeh.org/en/latest/docs/reference/palettes.html#bokeh-palettes
Bokeh Large Palettes:
    Greys256
    Inferno256
    Magma256
    Plasma256
    Viridis256
    Cividis256
    Turbo256

In case you want to update colorbar to different palette: 
    https://stackoverflow.com/questions/54948462/change-colormapper-via-callback-js-in-standalone-bokeh-plot
"""
colormap_mapper = {
    # 'DOXY_CTD': 'Magma256', 
    # 'DENS_CTD': 'Viridis256',
    'DOXY_CTD': 'Turbo256', 
    'DENS_CTD': 'Turbo256',
}

color_mapper = {}
for i, para in enumerate(DEFAULT_PARAMETERS):
    min_value = min(ctd_datasource.data[para])
    max_value = max(ctd_datasource.data[para])
    if min(mvp_datasource.data[para]) < min_value:
        min_value = min(mvp_datasource.data[para])
    if max(mvp_datasource.data[para]) > max_value:
        max_value = max(mvp_datasource.data[para])
    print(colormap_mapper[para])
    color_mapper[para] = LinearColorMapper(
        palette=colormap_mapper[para], 
        low=min_value, 
        high=max_value
    )
    if i == 0:
        # dummy_cm: not ideal, but it works..
        dummy_cm = LinearColorMapper(
            palette='Turbo256', 
            low=min_value, 
            high=max_value
        )

Turbo256
Turbo256


In [5]:
fig_map = get_map(
    title='Data spots', 
    x_range=(1400000, 2200000), 
    y_range=(7200000, 8500000),
    tooltips=[("Time", "@timestring")]
)
fig_map.xaxis.axis_label = 'Latitude'
fig_map.yaxis.axis_label = 'Longitude'

map_renderer_ctd = fig_map.circle(
    x="lon", 
    y="lat",
    size=6,
    color='blue',
    alpha=0.5,
    legend_label='CTD',
    source=ctd_pos_source,
)
map_renderer_mvp = fig_map.circle(
    x="lon", 
    y="lat",
    size=6,
    color='orange',
    alpha=0.5,
    legend_label='MVP',
    source=mvp_pos_source,
)
fig_map.legend.location = "top_left"

nonselected_circle = Circle(fill_alpha=0.3, line_color='grey')
map_renderer_ctd.nonselection_glyph = nonselected_circle
map_renderer_mvp.nonselection_glyph = nonselected_circle

In [6]:
color_bar = ColorBar(
    # color_mapper=color_mapper[DEFAULT_PARAMETERS[0]],
    color_mapper=dummy_cm,
    label_standoff=14,
    location=(0,0),
    title=''
)

transect = figure(
    width=800, height=300,
    x_axis_type='datetime',
    title='Selected transect parameter',
    tools='pan,wheel_zoom,box_select,reset,save',
    active_scroll="wheel_zoom",
)
transect.xaxis.axis_label = 'timestamp'
transect.yaxis.axis_label = 'PRES_CTD'
transect.y_range.flipped = True
transect_render = transect.circle(
    x="x", 
    y="y",
    fill_color={'field': 'z', 'transform': dummy_cm},
    line_color=None,
    # marker="square",
    size=8,
    alpha=1,
    source=profiles_mvp_source,
)

transect.add_layout(color_bar, 'right')

In [7]:
cm_map_callback = update_colormapper_transect(fig=transect, plot=transect_render, color_mapper=color_mapper)
z_sel = Select(value=DEFAULT_PARAMETERS[0], options=DEFAULT_PARAMETERS, title='Transect-parameter')
z_sel.js_on_change("value", cm_map_callback)
color_range_callback = update_colormapper_range(fig=transect, plot=transect_render, data_source=profiles_mvp_source)


In [8]:
TOOLTIPS = [("Pressure", "@y"), ("Value", "@z")]
profiles = figure(
    width=500, height=500,
    title='Selected paramter for CTD and MVP',
    tools='pan,wheel_zoom,box_select,reset,save',
    active_scroll="wheel_zoom",
    tooltips=TOOLTIPS
)
profiles.xaxis.axis_label = ""  # DEFAULT_PARAMETERS[0]
profiles.yaxis.axis_label = 'Pressure (dbar)'
profiles.y_range.flipped = True

profiles.circle(
    x="z",  # z column
    y="y",
    size=3,
    color='blue',
    alpha=0.5,
    legend_label='CTD',
    source=profiles_ctd_source,
)
profiles.circle(
    x="z",  # z column
    y="y",
    size=3,
    color='orange',
    alpha=0.5,
    legend_label='MVP',
    source=profiles_mvp_source,
)
profiles.legend.location = "top_left"

In [9]:
corr = figure(
    width=500, height=500,
    title='Selected parameter correlation CTD-MVP',
    tools='pan,wheel_zoom,box_select,reset,save',
    active_scroll="wheel_zoom",
)
corr.xaxis.axis_label = 'CTD'
corr.yaxis.axis_label = 'MVP'
corr_source = ColumnDataSource(data=dict(x=[], y=[]))
line_source = ColumnDataSource(data=dict(x=[], y=[]))
corr.line(
    x="x",
    y="y",
    color='black',
    source=line_source,
)
corr.circle(
    x="x",
    y="y",
    size=3,
    alpha=0.5,
    source=corr_source,
)

GlyphRenderer(id='1242', ...)

In [10]:
selection_change_mvp = lasso_transect_callback(
    z_selector=z_sel,
    pos_source=mvp_pos_source,
    data_source=mvp_datasource,
    plot_source=profiles_mvp_source,
)
mvp_pos_source.selected.js_on_change('indices', selection_change_mvp, color_range_callback)

selection_change_ctd = lasso_transect_callback(
    z_selector=z_sel,
    pos_source=ctd_pos_source,
    data_source=ctd_datasource,
    plot_source=profiles_ctd_source,
)
selection_correlation = lasso_correlation_callback(
    z_selector=z_sel,
    pos_source=ctd_pos_source,
    data_source=matching_datasource,
    plot_source=corr_source,
    line_source=line_source,
)
ctd_pos_source.selected.js_on_change('indices', selection_change_ctd, selection_correlation)

In [12]:
show(row(column(fig_map, transect), column(profiles, z_sel), column(corr)))
# show(row(column(fig_map, transect), column(z_sel)))
# show(row(column(fig_map, z_sel), column(profiles), column(corr)))